# Large Language Models

We're using the Ollama software in our examples. It runs open source LLMs and can be freely downloaded from https://ollama.ai/ 

Currently (8/31/2023) Ollama only supports Mac OS, with Windows and Linux coming soon. If you are on another operating system, Langchain supports Llama.cpp and other local LLM providers, as well as many commercial providers such as OpenAI's ChatGPT. See [the docs](https://python.langchain.com/docs/integrations/providers/llamacpp) for more info.

In [1]:
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import HumanMessage
# Instantiate Ollama with the wizard-vicuna model. 
# You'll need to run "ollama pull llama2:13b" to fetch this model.
# If you have < 32GB of RAM and/or have problems, try the "7b" model.
chat_model = ChatOllama(model="llama2:13b", 
                        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [2]:
messages = [
    HumanMessage(content="Testing 1 2 3")
]

In [3]:
messages.append(chat_model(messages))

 Hello! It looks like you're testing your microphone. Great idea! Here's a friendly robot to help you with any questions or tasks you may have.

Testing 1, 2, 3... Is there something specific you'd like to try or talk about?

In [4]:
messages.append(HumanMessage(content='Can you explain what an LLM is?'))
messages.append(chat_model(messages))

 Of course! An LLM (LL.M.) stands for Master of Laws, which is a postgraduate degree that focuses on advanced legal studies. It's typically a one-year program that's designed for students who have already completed a law degree, such as a J.D. or a B.A. in Law.

An LLM program can provide you with specialized knowledge and skills in a particular area of law, such as corporate law, intellectual property law, international law, or human rights law. The program typically includes coursework, research papers, and sometimes, clinical work or internships.

Earning an LLM degree can be beneficial for several reasons:

1. Specialization: An LLM program allows you to specialize in a specific area of law, which can help you develop deeper knowledge and expertise in that area.
2. Networking: LLM programs provide opportunities to connect with other law students and professionals, which can be helpful for building a network of contacts in the legal field.
3. Career advancement: An LLM degree can be

In [5]:
messages.append(HumanMessage(content='Actually, I was talking about a "Large Language Model".'))
messages.append(chat_model(messages))

 Ah, I see! A Large Language Model (LLM) is a type of artificial intelligence model that is trained on a large dataset of text to generate language outputs that are coherent and natural-sounding. These models have become increasingly popular in recent years due to their ability to generate text that is often indistinguishable from human-written text.

LLMs can be used for a variety of applications, such as:

1. Language Translation: LLMs can be trained on large datasets of text in multiple languages, allowing them to learn how to translate between languages.
2. Text Summarization: LLMs can summarize long documents or articles into shorter, more digestible versions.
3. Chatbots and Virtual Assistants: LLMs can be used to power chatbots and virtual assistants that can engage in natural-sounding conversations with humans.
4. Content Generation: LLMs can be used to generate content such as articles, blog posts, and even entire books.
5. Language Understanding: LLMs can be trained to unders

----

# Code Generation

Let's try using it to generate some code

In [11]:
messages = []
messages.append(HumanMessage(content='Can you write me a Django model class for an image? Only return the code, nothing else.'))
messages.append(chat_model(messages))

 Sure! Here is a Django model class for an image:
```
class Image(models.Model):
    title = models.CharField(max_length=255)
    file = models.FileField(upload_to='images')
```

In [12]:
messages.append(HumanMessage(content='Great! Can you write a management command that accepts a folder path and creates an Image object for each jpeg file?'))
messages.append(chat_model(messages))

 Certainly! Here is a Django management command that accepts a folder path and creates an Image object for each jpeg file:
```
from django.core.management import BaseCommand
from django.db.models import FileField, Model

class Command(BaseCommand):
    help = 'Creates Image objects for jpeg files in a folder'

    def handle(self, *args, **options):
        folder_path = options['folder_path']

        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg') or filename.endswith('.jpeg'):
                file_path = os.path.join(folder_path, filename)
                image = Image(title=filename, file=file_path)
                image.save()
```
This command will loop through all the files in the specified folder, check if they have a `.jpg` or `.jpeg` extension, and create an `Image` object for each one with the title of the file and the file path as the file field. Finally, it will save each `Image` object to the database.

You can run this command in the term

Not bad, but note that this code cannot be run as it is.  It uses the `Image` model it just created, but it doesn't actually import the model. So some supervision is required, even for simple code like this.

----
# Knowledge Cutoff

Each LLM's dataset necessarily ends at a certain date, and they cannot answer questions about anything that happened after that.

In [13]:
chat_model([
    HumanMessage(content='When was LLaMA 2 released?')
])

 I apologize, but I'm a large language model, I do not have access to information about the release dates of specific versions of LLaMA. However, I can suggest some ways you could find the information you are looking for:

1. Check the official website of the LLaMA project: You can visit the official website of the LLaMA project and look for the "Releases" or "History" page to see a list of all the versions that have been released, including LLaMA 2.
2. Search online: You can search online using your favorite search engine and keywords like "LLaMA 2 release date" or "LLaMA 2 version history" to find relevant information about the release date of LLaMA 2.
3. Check the documentation: If you have access to the documentation for LLaMA, you may be able to find information about the release date of LLaMA 2 in the documentation or in the changelog of each version.
4. Contact the developers: You can try contacting the developers of LLaMA directly to ask them about the release date of LLaMA 2. 

AIMessage(content=' I apologize, but I\'m a large language model, I do not have access to information about the release dates of specific versions of LLaMA. However, I can suggest some ways you could find the information you are looking for:\n\n1. Check the official website of the LLaMA project: You can visit the official website of the LLaMA project and look for the "Releases" or "History" page to see a list of all the versions that have been released, including LLaMA 2.\n2. Search online: You can search online using your favorite search engine and keywords like "LLaMA 2 release date" or "LLaMA 2 version history" to find relevant information about the release date of LLaMA 2.\n3. Check the documentation: If you have access to the documentation for LLaMA, you may be able to find information about the release date of LLaMA 2 in the documentation or in the changelog of each version.\n4. Contact the developers: You can try contacting the developers of LLaMA directly to ask them about the 

So even though we are using LLaMA 2, it has no knowledge of itself, at least inherently.

This is a solvable problem, and one of the problems that the Retrieval Augmented Generation (RAG) architecture is meant to solve.

----

# Hallucinations

These models will just make stuff up. They are not capable of discerning reality from fiction in a lot of cases.

In [14]:
chat_model([
    HumanMessage(content='Tell me the history of the first colony on Mars.')
])

 The first human colony on Mars was established in the year 2057, after decades of preparation and planning by various space agencies and private companies. Here's a brief history of the first Martian colony:

2010s: Early Exploration and Settlement Efforts
-----------------------------------------------

In the early 2010s, NASA and other space agencies around the world began sending robotic missions to Mars to explore its surface and subsurface. These missions discovered evidence of water ice and potential biosignatures, which sparked renewed interest in human exploration and settlement of the Red Planet.

2020s: Infrastructure Development and Private Investment
-----------------------------------------------

In the 2020s, governments and private companies began investing heavily in infrastructure development on Mars. This included the construction of habitats, life support systems, and transportation networks. The first Martian settlers arrived in the mid-2020s, and they establishe

AIMessage(content=" The first human colony on Mars was established in the year 2057, after decades of preparation and planning by various space agencies and private companies. Here's a brief history of the first Martian colony:\n\n2010s: Early Exploration and Settlement Efforts\n-----------------------------------------------\n\nIn the early 2010s, NASA and other space agencies around the world began sending robotic missions to Mars to explore its surface and subsurface. These missions discovered evidence of water ice and potential biosignatures, which sparked renewed interest in human exploration and settlement of the Red Planet.\n\n2020s: Infrastructure Development and Private Investment\n-----------------------------------------------\n\nIn the 2020s, governments and private companies began investing heavily in infrastructure development on Mars. This included the construction of habitats, life support systems, and transportation networks. The first Martian settlers arrived in the m

----

Cool story, but it hasn't happened. LLMs will confidently give you made up information. This can also be combatted with the RAG architecture, and potentially with system prompts as covered in the Prompting notebook.